In [25]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tensorflow
import tflearn
import random
import json
import pickle
from keras.models import Sequential
from keras.layers import Dense

with open('./Datasets/intents.json') as file:
    data = json.load(file)

try:
    with open('data.pickle','rb') as f:
        words,labels,training, output = pickle.load(f)
except:
    words = []
    labels =[]
    docs_x = []
    docs_y = []

# stemming takes each word in out patters and get to its root, stemming eliminates extra characters and helps make model more accurate
    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrd = nltk.word_tokenize(pattern)
            words.extend(wrd) # wrds is already a list so rather than appending through it and appending one by one we can append the entire list
            docs_x.append(wrd)
            docs_y.append(intent['tag'])
        if intent['tag'] not in labels:
            labels.append(intent['tag'])
        
    words= [stemmer.stem(w.lower()) for w in words if w!= '?']
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty=[ 0 for _ in range(len(labels))]   #classes to labels

    for x,doc in enumerate(docs_x):
        bag =[]
    
        wrds = [stemmer.stem(w.lower()) for w in doc]
    
        for w in words:
            if  w in wrds:
                bag.append(1)
            else:
                bag.append(0)
            
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])]=1
    
        training.append(bag)
        output.append(output_row)
    
    training = np.array(training)
    output = np.array(output)
    
    with open('data.pickle','wb') as f:
        pickle.dump((words,labels,training, output),f) 


tensorflow.reset_default_graph()

net = tflearn.input_data(shape =[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(output[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load('model.tflearn')
except:
    
    model.fit(training, output,n_epoch=1000, batch_size=8, show_metric=True)
    model.save('chatbotV1.tflearn')
    


In [21]:
#model = Sequential()
#model.add(Dense(8, input_dim=len(training[0]), activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(len(output[0]), activation='softmax'))

Training Step: 12999  | total loss: 1.19373 | time: 0.065s
| Adam | epoch: 1000 | loss: 1.19373 - acc: 0.4749 -- iter: 096/104
Training Step: 13000  | total loss: 1.22391 | time: 0.070s
| Adam | epoch: 1000 | loss: 1.22391 - acc: 0.4524 -- iter: 104/104
--
INFO:tensorflow:C:\Users\aditya\Desktop\Granthium_ChatBot\chatbotV1.tflearn is not in all_model_checkpoint_paths. Manually adding it.
